# Sistema de recomendación de comidas y establecimientos según restaurant deseado

## Modelamiento y evaluación con machine learning

### 1. Objetivos:
Crear una tabla que sirva para la función 1 del sistema de sistema de recomendación. Esta tabla sera exportada econ el nombre modelo3.parquet.

### 2. Importación de libreria:

In [ ]:
import numpy as np
import pandas as pd
import spacy
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ndcg_score
from sklearn.metrics import ndcg_score
from sklearn.model_selection import KFold
import random 
import seaborn as sns
import matplotlib.pyplot as plt

### 3. Carga y visualización del DataFrame

In [ ]:
df_business = pd.read_parquet('Anexos/Datos_en_tablas/business.parquet')
df_business_categories = pd.read_parquet('Anexos/Datos_en_tablas/business_categories.parquet')
df_categories = pd.read_parquet('Anexos/Datos_en_tablas/categories.parquet')
df_full_categories = pd.read_parquet('Anexos/Datos_en_tablas/full_categories.parquet')
df_cities = pd.read_parquet('Anexos/Datos_en_tablas/cities.parquet')
df_states = pd.read_parquet('Anexos/Datos_en_tablas/states.parquet')
df_users = pd.read_parquet('Anexos/Datos_en_tablas/users.parquet')
df_reviews = pd.read_parquet('Anexos/Datos_en_tablas/reviews.parquet')
df_ml_2 = pd.read_parquet('Anexos/Datos_en_tablas/modelo2.parquet')

In [ ]:
df_ml_2.columns

Index(['id_business', 'avg_rating', 'business_name', 'business_address',
       'hours', 'category_name', 'city_name', 'review_count', 'region',
       'key_ingredient', 'description'],
      dtype='object')

#### 4. Creación de un DataFrame df_ml_3 para crear el sistema de recomendación

In [ ]:
df_users.rename(columns={'id': 'id_user'}, inplace=True)

# Realizar los merges
df_ml_3 = (df_ml_2.merge(df_reviews, on='id_business', how='inner')
                .merge(df_users, on='id_user', how='inner'))

In [ ]:
df_ml_3

,id_business,avg_rating,business_name,business_address,hours,category_name,city_name,review_count,region,key_ingredient,description,id,id_user,rating,text,date,origin
0,1,5.0,Wawa,870 2nd St Pike,"{'Friday': '0:0-0:0', 'Monday': '0:0-0:0', 'Sa...",Breakfast restaurant,Richboro,2,North-America,"Eggs, Pancakes, Bacon, Sausage, Toast","Breakfast restaurant Eggs, Pancakes, Bacon, Sa...",1475725,108982599491435635920,5.0,It’s Wawa what else is there to say,2021-03-08 13:44:20.056,G
1,1,5.0,Wawa,870 2nd St Pike,"{'Friday': '0:0-0:0', 'Monday': '0:0-0:0', 'Sa...",Breakfast restaurant,Richboro,2,North-America,"Eggs, Pancakes, Bacon, Sausage, Toast","Breakfast restaurant Eggs, Pancakes, Bacon, Sa...",1475726,116523048530310716264,1.0,Tons of people shopping without masks and no o...,2020-08-20 00:39:42.336,G
2,1,5.0,Wawa,870 2nd St Pike,"{'Friday': '0:0-0:0', 'Monday': '0:0-0:0', 'Sa...",Breakfast restaurant,Richboro,2,North-America,"Eggs, Pancakes, Bacon, Sausage, Toast","Breakfast restaurant Eggs, Pancakes, Bacon, Sa...",1475727,103289463208586885687,4.0,This one is a newer location with Gas. Inside...,2019-12-03 20:54:25.459,G
3,1,5.0,Wawa,870 2nd St Pike,"{'Friday': '0:0-0:0', 'Monday': '0:0-0:0', 'Sa...",Breakfast restaurant,Richboro,2,North-America,"Eggs, Pancakes, Bacon, Sausage, Toast","Breakfast restaurant Eggs, Pancakes, Bacon, Sa...",1475728,115245582389789776884,5.0,Wawa is a staple gas station for any native of...,2020-01-16 16:17:05.822,G
4,1,5.0,Wawa,870 2nd St Pike,"{'Friday': '0:0-0:0', 'Monday': '0:0-0:0', 'Sa...",Breakfast restaurant,Richboro,2,North-America,"Eggs, Pancakes, Bacon, Sausage, Toast","Breakfast restaurant Eggs, Pancakes, Bacon, Sa...",1475729,107063662590006636869,1.0,Had to wait 15 minutes for my 2 drinks. This i...,2019-06-07 22:58:57.531,G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7923385,9999,4.5,La Conca D'oro,440 Main St,"{'Friday': '4–9PM', 'Monday': '4–9PM', 'Saturd...",Italian restaurant,Jefferson Heights,4,Europe,"Pasta, Tomato Sauce, Olive Oil, Garlic, Basil","Italian restaurant Pasta, Tomato Sauce, Olive ...",1249656,109376293637242608974,5.0,None,2020-02-14 01:57:03.099,G
7923386,9999,4.5,La Conca D'oro,440 Main St,"{'Friday': '4–9PM', 'Monday': '4–9PM', 'Saturd...",Italian restaurant,Jefferson Heights,4,Europe,"Pasta, Tomato Sauce, Olive Oil, Garlic, Basil","Italian restaurant Pasta, Tomato Sauce, Olive ...",1249657,115093240521874554718,5.0,None,2019-03-28 03:44:02.601,G
7923387,9999,4.5,La Conca D'oro,440 Main St,"{'Friday': '4–9PM', 'Monday': '4–9PM', 'Saturd...",Italian restaurant,Jefferson Heights,4,Europe,"Pasta, Tomato Sauce, Olive Oil, Garlic, Basil","Italian restaurant Pasta, Tomato Sauce, Olive ...",1249658,117373888308393053067,5.0,None,2020-08-30 16:38:22.328,G
7923388,9999,4.5,La Conca D'oro,440 Main St,"{'Friday': '4–9PM', 'Monday': '4–9PM', 'Saturd...",Italian restaurant,Jefferson Heights,4,Europe,"Pasta, Tomato Sauce, Olive Oil, Garlic, Basil","Italian restaurant Pasta, Tomato Sauce, Olive ...",1249659,108035610841854471792,4.0,None,2018-10-08 22:49:36.726,G


In [ ]:
df_ml_3.columns

Index(['id_business', 'avg_rating', 'business_name', 'business_address',
       'hours', 'category_name', 'city_name', 'review_count', 'region',
       'key_ingredient', 'description', 'id', 'id_user', 'rating', 'text',
       'date', 'origin'],
      dtype='object')

In [ ]:
df_ml_3['weighted_rating'] = df_ml_3['avg_rating'] * np.log(1 + df_ml_3['review_count'])

In [ ]:
df_ml_3.columns

Index(['id_business', 'avg_rating', 'business_name', 'business_address',
       'hours', 'category_name', 'city_name', 'review_count', 'region',
       'key_ingredient', 'description', 'id', 'id_user', 'rating', 'text',
       'date', 'origin', 'weighted_rating'],
      dtype='object')

In [ ]:
df_ml_3 = df_ml_3[['id_business', 'business_name', 'category_name', 'city_name', 'business_address', 'region', 'description', 'avg_rating', 'weighted_rating', 'date', 'hours', 'id_user']]

In [ ]:
df_ml_3

,id_business,business_name,category_name,city_name,business_address,region,description,avg_rating,weighted_rating,date,hours,id_user
0,1,Wawa,Breakfast restaurant,Richboro,870 2nd St Pike,North-America,"Breakfast restaurant Eggs, Pancakes, Bacon, Sa...",5.0,5.493061,2021-03-08 13:44:20.056,"{'Friday': '0:0-0:0', 'Monday': '0:0-0:0', 'Sa...",108982599491435635920
1,1,Wawa,Breakfast restaurant,Richboro,870 2nd St Pike,North-America,"Breakfast restaurant Eggs, Pancakes, Bacon, Sa...",5.0,5.493061,2020-08-20 00:39:42.336,"{'Friday': '0:0-0:0', 'Monday': '0:0-0:0', 'Sa...",116523048530310716264
2,1,Wawa,Breakfast restaurant,Richboro,870 2nd St Pike,North-America,"Breakfast restaurant Eggs, Pancakes, Bacon, Sa...",5.0,5.493061,2019-12-03 20:54:25.459,"{'Friday': '0:0-0:0', 'Monday': '0:0-0:0', 'Sa...",103289463208586885687
3,1,Wawa,Breakfast restaurant,Richboro,870 2nd St Pike,North-America,"Breakfast restaurant Eggs, Pancakes, Bacon, Sa...",5.0,5.493061,2020-01-16 16:17:05.822,"{'Friday': '0:0-0:0', 'Monday': '0:0-0:0', 'Sa...",115245582389789776884
4,1,Wawa,Breakfast restaurant,Richboro,870 2nd St Pike,North-America,"Breakfast restaurant Eggs, Pancakes, Bacon, Sa...",5.0,5.493061,2019-06-07 22:58:57.531,"{'Friday': '0:0-0:0', 'Monday': '0:0-0:0', 'Sa...",107063662590006636869
...,...,...,...,...,...,...,...,...,...,...,...,...
7923385,9999,La Conca D'oro,Italian restaurant,Jefferson Heights,440 Main St,Europe,"Italian restaurant Pasta, Tomato Sauce, Olive ...",4.5,7.242471,2020-02-14 01:57:03.099,"{'Friday': '4–9PM', 'Monday': '4–9PM', 'Saturd...",109376293637242608974
7923386,9999,La Conca D'oro,Italian restaurant,Jefferson Heights,440 Main St,Europe,"Italian restaurant Pasta, Tomato Sauce, Olive ...",4.5,7.242471,2019-03-28 03:44:02.601,"{'Friday': '4–9PM', 'Monday': '4–9PM', 'Saturd...",115093240521874554718
7923387,9999,La Conca D'oro,Italian restaurant,Jefferson Heights,440 Main St,Europe,"Italian restaurant Pasta, Tomato Sauce, Olive ...",4.5,7.242471,2020-08-30 16:38:22.328,"{'Friday': '4–9PM', 'Monday': '4–9PM', 'Saturd...",117373888308393053067
7923388,9999,La Conca D'oro,Italian restaurant,Jefferson Heights,440 Main St,Europe,"Italian restaurant Pasta, Tomato Sauce, Olive ...",4.5,7.242471,2018-10-08 22:49:36.726,"{'Friday': '4–9PM', 'Monday': '4–9PM', 'Saturd...",108035610841854471792


In [ ]:
df_ml_3.to_parquet('Anexos/Datos_en_tablas/modelo3.parquet')